# Ejercicio 2

En el siguiente problema, se presenta un conjunto de datos correspondientes a escritos de Shakespear. El objetivo del problema es crear un modelo capaz de generar texto con dialecto de época y escritura en verso y prosa.

# Librerías y entorno

In [ ]:
# Install tensorflow 2.15
!pip install tensorflow==2.15.0

In [ ]:
!pip install googletrans==4.0.0-rc1

In [ ]:
import requests

import tensorflow as tf
import numpy as np
import os
import time
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.preprocessing.text import Tokenizer

from googletrans import Translator


In [ ]:
translator = Translator()

In [ ]:
# Configurar para que TensorFlow utilice la GPU por defecto
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Configurar para que TensorFlow asigne memoria dinámicamente
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        # Especificar la GPU por defecto
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Manejar error
        print(e)

1 Physical GPUs, 1 Logical GPUs


# Dataset


In [ ]:
# URL del dataset
url = "https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt"

# Realizar la solicitud de descarga
response = requests.get(url)

# Guardar el archivo
with open("shakespeare.txt", "wb") as file:
    file.write(response.content)

print("Descarga completada.")


Descarga completada.


In [ ]:
# Read, then decode for py2 compat.
text = open("shakespeare.txt", 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [ ]:
# Take a look at the first 2000 characters in text
print(text[:2000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [ ]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


# Modelo carácter a carácter

## Preprocesamiento

### Vectorización

Previo al entrenamiento, necesitamos convertir el texto a una representacion numerica.

La capa tf.keras.layers.StringLookup nos permite convertir cada caracter en un ID numerico. Solo necesita que el texto este separado primero en tokens.

A la par de hacerlo para nuestro texto lo mostramos con un ejemplo, para entender cómo funciona

In [ ]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Ahora creamos la capa tf.keras.layers.StringLookup:

In [ ]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

Esto nos convierte de tokens a IDs de caracteres:

In [ ]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

Dado que el proposito de este ejercicio es generar texto, tambien sera importante invertir esta representacion y recuperar texto legible desde estos IDs. Para esto utilizamos `tf.keras.layers.StringLookup(..., invert=True).`

Nota: Aquí, en lugar de pasar el vocabulario original generado con `sorted(set(text))`, usamos el método `get_vocabulary()` de la capa `tf.keras.layers.StringLookup` para que los tokens `[UNK]` se configuren de la misma manera.

In [ ]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

Esta capa recupera los caracteres desde los vectores de IDs y los retorna como un `tf.RaggedTensor` de caracteres:



In [ ]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Finalmente usando `tf.strings.reduce_join` se pueden volver a juntar los caracteres en texto.

In [ ]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [ ]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [ ]:
# Convertimos el texto en ids numéricos
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


## Predicción carácter a carácter


Dado un caracter, o una secuencia de caracteres, ¿cuál es el siguiente caracter más probable? Esta es la tarea para la que estamos entrenando al modelo. La entrada al modelo será una secuencia de caracteres y entrenamos el modelo para predecir la salida: el siguiente carácter en cada paso de tiempo.

Dado que los RNN mantienen un estado interno que depende de los elementos vistos anteriormente, a partir de todos los caracteres calculados hasta este momento, ¿cuál es el siguiente carácter?

## Crear los ejemplos de entrenamiento

Dividimos el texto en secuencias de ejemplo. Cada secuencia de entrada contendrá `seq_length` caracteres del texto.

Para cada secuencia de entrada, los targets correspondientes contienen la misma longitud de texto, excepto que se desplazan un carácter hacia la derecha.

Así que divida el texto en fragmentos de `seq_length+1`. Por ejemplo, digamos `que seq_length` es 3 y nuestro texto es "Hola". La secuencia de entrada sería "Hol" y la secuencia target "ola".

Para hacer esto, usamos la función `tf.data.Dataset.from_tensor_slices` para convertir el vector de texto en una secuencia de índices de caracteres.

In [ ]:
seq_length = 100

El método `batch` nos permite convertir fácilmente estos caracteres individuales en secuencias del tamaño deseado.



In [ ]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


Es mas facil ver lo que esta haciendo si unimos de vuelta los tokens en texto:

In [ ]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


Para el entrenamiento, necesitaremos un conjunto de datos de pares `(input, label)`. Donde `input` y `label` son secuencias. En cada timestep, la entrada es el carácter actual y la etiqueta es el siguiente carácter.

Aquí hay una función que toma una secuencia como entrada, la duplica y la desplaza para alinear la entrada y la etiqueta para cada timestep:

In [ ]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
# Probamos el método de arriba
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [ ]:
dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


## Batches de entrenamiento

Usamos `tf.data` para dividir el texto en secuencias manejables. Pero antes de introducir estos datos en el modelo, es necesario mezclarlos y batchearlos.

In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

## Construccion del modelo


Este modelo tiene tres capas:

* `tf.keras.layers.Embedding`: La capa de entrada. Una lookup table entrenable que asignará cada ID de carácter a un vector con dimensiones `embedding_dim`;
* `tf.keras.layers.GRU`: una capa recurrente GRU de tamaño units=rnn_units (también se puede usar una capa LSTM aquí).
* `tf.keras.layers.Dense`: La capa de salida, con salidas` vocab_size`. Genera un logit para cada carácter del vocabulario. Estas son las probabilidades de cada caracter según el modelo.

In [ ]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [ ]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

Por cada caracter el modelo calcula su embedding, corre la GRU un timestep con el embedding como entrada y aplica la capa densa para generar los logits prediciendo la probabilidades del siguiente caracter.

## Probar el modelo

Ejecutamos el modelo para ver que se comporta como se esperaba.

Primero verificamos la shape de salida:

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


En el ejemplo anterior, la longitud de la secuencia de la entrada es 100, pero el modelo se puede ejecutar con entradas de cualquier longitud:

In [ ]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Para obtener predicciones reales del modelo, se deben tomar muestras de la distribución de salida para obtener índices de caracteres reales. Esta distribución está definida por los logits sobre el vocabulario de los caracteres.

Nota: Es importante tomar una muestra de esta distribución, ya que tomar el argmax de la distribución puede fácilmente hacer que el modelo se atasque en un bucle.

Tomando como ejemplo el primero del batch:

In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

Esto nos da para cada timestep una predicción del siguiente índice de caracteres:



In [ ]:
sampled_indices


array([ 0, 26, 57, 23, 27, 12,  2, 50, 58, 65, 44, 22,  6, 34, 42, 42,  6,
        2, 61, 11, 11, 15, 65, 31, 22, 30, 39,  2,  8, 65, 24, 34, 46, 16,
       25, 13, 46, 64, 32, 56,  3, 60, 65, 14, 31, 21,  2, 58, 42, 11, 26,
       43, 31, 23, 23, 20, 31, 46, 58, 60, 51, 46, 40, 57, 47, 42, 63, 12,
       55,  4, 37, 11,  5, 37, 60, 52, 16, 46, 54, 19, 42, 60, 25,  1, 65,
        3, 12, 35, 41, 40, 53, 12, 43,  2,  4,  1, 23, 27, 63, 65])

In [ ]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b"bring not comfort home,\nThey'll give him death by inches.\n\nSICINIUS:\nWhat's the news?\n\nSecond Messen"

Next Char Predictions:
 b"[UNK]MrJN; kszeI'Ucc' v::BzRIQZ -zKUgCL?gySq!uzARH sc:MdRJJGRgsulgarhcx;p$X:&XumCgoFcuL\nz!;Vban;d $\nJNxz"


## Entrenamiento del modelo

El problema puede tratarse como un problema de clasificación estándar. Dado el estado RNN anterior y la entrada en este timestep, predice la clase del siguiente carácter.



**Agregamos un optimizador y una funcion costo**

La función de pérdida estándar `tf.keras.losses.sparse_categorical_crossentropy` funciona en este caso porque se aplica en la última dimensión de las predicciones.

Debido a que su modelo devuelve logits, necesita configurar el indicador `from_logits`.

In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)


In [ ]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.190633, shape=(), dtype=float32)


Un modelo recién inicializado no debería estar demasiado seguro de sí mismo, todos los logits de salida deberían tener magnitudes similares. Para confirmar esto, puede comprobar que la exponencial del costo medio es aproximadamente igual al tamaño del vocabulario. Una pérdida mucho mayor significa que el modelo está seguro de sus respuestas incorrectas y está mal inicializado:

In [ ]:
tf.exp(example_batch_mean_loss).numpy()


66.06458

Compilamos el modelo con `tf.keras.Model.compile` indicando el optimizador y la funcion costo:



In [ ]:
model.compile(optimizer='adam', loss=loss)


**Checkpoints del modelo**

Usamos el callback `tf.keras.callbacks.ModelCheckpoint` para que se guarden checkpoints del modelo durante el entrenamiento.

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

**Ejecucion del entrenamiento**

Primero entrenamos con 20 épocas, al no obtener buenos resultos, decimos utilizar 40.

En el entrenamiento con 20 épocas notamos que el texto no era coherente.

In [ ]:
EPOCHS = 40

In [ ]:
tf.config.run_functions_eagerly(True)


In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/40
172/172 [==============================] - 15s 71ms/step - loss: 2.7212
Epoch 2/40
172/172 [==============================] - 14s 74ms/step - loss: 1.9940
Epoch 3/40
172/172 [==============================] - 14s 73ms/step - loss: 1.7153
Epoch 4/40
172/172 [==============================] - 16s 77ms/step - loss: 1.5545
Epoch 5/40
172/172 [==============================] - 15s 76ms/step - loss: 1.4540
Epoch 6/40
172/172 [==============================] - 15s 76ms/step - loss: 1.3865
Epoch 7/40
172/172 [==============================] - 15s 75ms/step - loss: 1.3346
Epoch 8/40
172/172 [==============================] - 14s 74ms/step - loss: 1.2896
Epoch 9/40
172/172 [==============================] - 14s 74ms/step - loss: 1.2489
Epoch 10/40
172/172 [==============================] - 15s 76ms/step - loss: 1.2095
Epoch 11/40
172/172 [==============================] - 14s 75ms/step - loss: 1.1688
Epoch 12/40
172/172 [==============================] - 14s 75ms/step - loss: 1.1282
E

## Generacion de texto

La forma más sencilla de generar texto con este modelo es ejecutarlo en un bucle y realizar un seguimiento del estado interno del modelo a medida que lo ejecutamos.



Cada vez que llamamos al modelo, pasamos algún texto y un estado interno. El modelo devuelve una predicción para el siguiente caracter y su nuevo estado. Vuelva a pasar la predicción y el estado para continuar generando texto.


Lo siguiente hace una predicción de un solo paso:

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Lo ejecutamos en un bucle para generar texto. Al observar el texto generado, veremos que el modelo sabe cuándo poner mayúsculas, hacer párrafos e imita un vocabulario de escritura similar al de Shakespear. Con el reducido número de épocas de entrenamiento, todavía no ha aprendido a formar frases coherentes, incluso primero probamos con 20 épocas y luego con 40 y notamos el mismo compartamiento.

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['Caius Marcius'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Caius Marcius?

BRUTUS:
It stands your grace, why should we fight?
Where is thy lapour need no means mailest us;
Me then we will open them. My cousin Juliet,
To lusting not himself with shried and entering
banish'd; though unskilf my budgets, care not so.

MENENIUS:
The son, whereof I give you, sir, thanks, and
Tread the King of heaven, the tongues droposors!
Where are thy tongue climbs us not one but death:
And when I give me leave to retire,
Or by the honourable roots be courted
A sentence of your breast for bearing but
Yelk in 's east, if they were equals;
'Twere all other forbid be a happy death:
'Tis bastard keen's; the tunes of death no puties for sense;
They tend the cause of my body's voices
Are now we can make heavy gnam; it will come to me,
In the ear that is about a lady, your
night, lords, going to find him stealth by:
but I had given me back the case of France
Hath pawnet out an open in our gentle highwes;
So far in blood, thou, idle weeds and kneel
but my death with child

Para poder interpretar mejor los resultados obtenidos, lo traducimos al español. La idea es poder analizar si el texto generado tiene coherencia.

In [ ]:
# Traducción al español
generated_text = result[0].numpy().decode('utf-8')
translated_text = translator.translate(generated_text, src='en', dest='es').text
print("Texto en español:\n")
print(translated_text, '\n\n' + '_'*80)

Texto en español:

¿Caius Marcius?

Brutus:
Era su nombre.Asalto
De Roma;'Es pero la alondra mañana
Y todas las cosas que pueden contemplarlos con mi corazón,
Sobre tu fatal cannertome empinado
Eso los hizo en la parte de su sonido obitiano:
Prefiero creo que este terreno
Como yo por confesado contra mi descanso
Mientras que me ponen a todos:
También lo es la envidia en la luna.

Northumberland:
¿Cuál fue tu deber para la muerte?¡Oh pecado!
¿Qué te da tanto ahora, con un poderoso tortillero que deberíamos?
Me desatan a otro.

Marcius:
Si el medio noble pudiera morir.

York:
Tengamos tu pompa, Malk;Su nombre es Tybalt, tú, mis señores,
Si la pena me puede hacer por sí misma, rico en desprecio,
Y te saqueó, acomoda a Juliut un mundo.Alack, digo.

Enfermero:
¡Amante!¡Oh!Estamos malcriados y la velocidad.

Pedante:
Jurar tú?Art tú eres mentira, en cuestión,
Ir a whipph;y, por el campo de Saint Alban,
La verdadera esperanza se apresuró al dolor de encuentro
Para desatrar su madre murió: Por

Además de aumentar las épocas también podríamos experimentar con una secuencia de inicio diferente, intentar agregar otra capa RNN para mejorar la precisión del modelo o ajustar el parámetro de temperatura para generar predicciones más o menos aleatorias.

Veremos si luego del modelo para predecir palabra a palabra agregamos algo de todo esto.

Si queremos que el modelo genere texto *más rápido*, lo más fácil que se puede hacer es generar el texto por batches. En el siguiente ejemplo, el modelo genera 5 resultados aproximadamente en el mismo tiempo que tomó generar 1 arriba.

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['Caius Marcius', 'Caius Marcius', 'Caius Marcius', 'Caius Marcius', 'Caius Marcius'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)


tf.Tensor(
[b"Caius Marcius?\n\nHORTENSIO:\nMadam, where lies Clubb'd with a bond?\n\nHASTINGS:\nMy lord!\n\nKING RICHARD III:\nSweet Kate, no, not the son o' the midst\nJetuen her fanth. Conceive a brother!\n\nISABELLA:\nWhy do you come?\n\nBoatswain:\nLook, you three, Margaret, Romeo was broke:\nWhich grieves next was a forterler.\n\nKING RICHARD III:\nStanley, look upon thee.\n\nForrecesain:\nI pray now, keep you uncannot be:\nO, she, your father tell me what thou learn'd,\nThat they are in a treaty. Time, lead ashame!\nI pray, sir, we have writ you?\n\nSecond Murderer:\n'Zounds, whom thou liest; his noble cousins may prove\nSomething hither come to see him and all chaff:\nDreaming suddensicle thirtus, which at\nlasting rebellion; and then indeed pluck\nhath made a poor cenvay for a king:\nCome, bravE Old nothing but sweeter's death,\n'I would here perceive this dead!\n\nGLOUCESTER:\nNor no one that do sit down.\n\nKING RICHARD III:\nSoldiers shall this body to my reputation.\nMake 

## Evaluación del modelo carácter a carácter

In [ ]:
def generate_text(model, start_string, num_generate=500, temperature=1.0):
    input_eval = ids_from_chars(tf.strings.unicode_split(start_string, 'UTF-8')).numpy()
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []

    for _ in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(chars_from_ids(predicted_id).numpy().decode('utf-8'))

    return start_string + ''.join(text_generated)


Seleccionamos diferentes temperaturas y longitudes para generar los textos

In [ ]:
# Generar fragmentos con diferentes configuraciones
start_string = "Caius Marcius"

# Fragmentos con temperatura = 1
print("Temperatura 1:")
for i in range(5):
    print(f"Fragmento {i+1}:")
    print(generate_text(model, start_string=start_string, num_generate=200, temperature=1.0))
    print()

# Fragmentos con temperatura < 1
print("Temperatura < 1 (0.5):")
for i in range(5):
    print(f"Fragmento {i+1}:")
    print(generate_text(model, start_string=start_string, num_generate=200, temperature=0.5))
    print()

# Fragmentos con temperatura > 1
print("Temperatura > 1 (1.5):")
for i in range(5):
    print(f"Fragmento {i+1}:")
    print(generate_text(model, start_string=start_string, num_generate=200, temperature=1.5))
    print()


Temperatura 1:
Fragmento 1:
Caius Marcius?
O:
Why bonco'd yous ur ber INIO haf tees y ichthtowhoushousthin,
Horer'th lioourn d
I'derg ak ar gllelf cu s s bora ange sck thend imous
Pam s
CA:

LAs veathestourtuthiedoour t ngererdor yow ayourst

Fragmento 2:
Caius Marcius?
SUSINTowe
VO:


Houlourthas, pren inena me, nd, whiend se abrs pranorthige, asive, h peirear'TELely,
Loupliverd in oungomily s;
Whithouckes thicheree,
Why.
CHiowin t th m.



F:
ME Detus youst apel 

Fragmento 3:
Caius Marcius?
VE:
CEn d pree man,

Londs smamyouthitovedese! urad I byouce w culds h blele bl s cl thesis, wh, s by s dst w ause wimanor me;




I f de toue thor theyofor;
Hee se, he t

Thal har aten paly w h why

Fragmento 4:
Caius Marcius?
Chith,
ORI housamoutes t
Thomy w ar!
NINTher,
HORKI t thouthinowe tharkistst bred tlete, thin t, tee s HE:


Pl-viseno bre llethey----ck I ivin g thit,

HESe th buthisp' ll thizers; h athare,

TE ho

Fragmento 5:
Caius Marcius?

Thed, banour thecondile.
I mais, LINCowif wh

Observaciones:
* Temperatura = 1.0

Genera texto con una mezcla moderada de creatividad y coherencia.
Las frases tienen cierta estructura gramatical pero contienen muchas palabras inventadas o incoherentes.

* Temperatura < 1.0 (0.5)

Las predicciones se vuelven más conservadoras. El texto tiende a repetir patrones comunes y generar frases más predecibles y monótonas.
Hay menos creatividad y se observan repeticiones frecuentes de palabras o estructuras.
Se observa muy poca choerencia en el texto generado.

* Temperatura > 1.0 (1.5)

Aumenta considerablemente la diversidad en el texto, pero a costa de la coherencia.
Se producen palabras y estructuras altamente aleatorias, que no tienen sentido.
La salida resulta caótica y menos útil para aplicaciones prácticas donde la coherencia es esencial.

En algunos casos podemos ver que el texto conserva mayúsculas donde corresponden y la forma de verso y prosa.

Probamos distintas longitudes de secuencia

In [ ]:
# Probamos con longitudes diferentes
longitudes = [100, 200, 500]

for length in longitudes:
    print(f"Fragmento con longitud {length}:")
    print(generate_text(model, start_string=start_string, num_generate=length, temperature=1.0))
    print()


Fragmento con longitud 100:
Caius Marcius?
Stho! touresit I hour oores f se ar alame, the ve;

Lougnd bers t ff a doust I e shatoucour d, thi

Fragmento con longitud 200:
Caius Marcius?
He' re the;

Honkeceindoun s n he bu
Chtonf orof mach t be! sthy bly, hureengees winof shand p nsowin:
Whepr, belllay grave w ous d; s amenghave Car--pr me mase t, IIEnthathomyousery Gll u uro anses

Fragmento con longitud 500:
Caius Marcius?
Tous here t.
HO:
fer th t Pe the berece thosor,
SAnoficu nglls ystathasivivindour h bllll nd.
CEO:
CEThonou'dagustely.
S beve bopad cken melotar'sifoupllerd oured vea thopoug Starse, thilloveas,
ARINGHARelss, bucensirde:
D couthesurou IN: f a byst, w qurajur me:
ICHat ds goromanther couryoschaismecigovofanthyoupr pours, hy gr winchers;
INThan t s, this; icord sass
CEOMy m coflll igmuroulinte wis h me aswnca llld ang, f CHare Pr?

NEgee htower lld bl-lithtuto aviestha haferst bourou nd tho be p



* Fragmento con longitud 100:
Observación: El texto es breve y presenta fragmentos de palabras y frases que se acercan más a un estilo coherente. La falta de contexto suficiente hace que no logre desarrollar una narrativa sólida.

Ejemplo destacado:

**"Caius Marcius? Stho! touresit I hour oores f se ar alame, the ve;"**

Aunque las palabras no tienen sentido completo, el uso de puntuación y estructura tiene cierta similitud con un diálogo dramático.


* Fragmento con longitud 200:
Aparecen más palabras conectadas y estructuras similares a frases de Sheaskpear. Sin embargo, al aumentar la longitud, el modelo comienza a introducir incoherencias más evidentes.

Ejemplo destacado:

**"Whepr, belllay grave w ous d; s amenghave Car--pr me mase t"**

Aunque incoherente, se observan patrones del estilo del texto de entramiento, como palabras truncadas y una estructura similar a un poema.

* Fragmento con longitud 500:
 Aunque logra mantener cierta consistencia estilística (uso de puntuación, extructura de poema), la narrativa se desarma rápidamente, y las incoherencias se vuelven evidentes.

Ejemplo destacado:

**"Tous here t. HO: fer th t Pe the berece thosor, SAnoficu nglls ystathasivivindour h bllll nd."**
El texto carece de sentido, pero mantiene un estilo que podría parecer inspirado en un diálogo dramático.

**Conclusión general:** A mayores longitudes, el modelo pierde su capacidad de mantener una estructura coherente, y las repeticiones, palabras inventadas y frases ininteligibles se incrementan.


# Modelo palabra a palabra

## Vectorización

Cambia de carácter a carácter, a palabra a palabra:

In [ ]:
words = text.split()  # Divide el texto en palabras usando espacios
print(f"Cantidad de palabras: {len(words)}")
print(f"Primeras 10 palabras: {words[:10]}")


Cantidad de palabras: 202651
Primeras 10 palabras: ['First', 'Citizen:', 'Before', 'we', 'proceed', 'any', 'further,', 'hear', 'me', 'speak.']


Mapeamos las palabras a IDs

In [ ]:

tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])  # Aprende el vocabulario
word_index = tokenizer.word_index  # Diccionario de palabras a IDs
sequences = tokenizer.texts_to_sequences([text])  # Convierte el texto a secuencias de IDs
sequences = sequences[0]  # Extrae la lista de IDs, ya que `texts_to_sequences` devuelve una lista de listas

print(f"Primeras 10 IDs: {sequences[:10]}")


Primeras 10 IDs: [88, 269, 139, 35, 969, 143, 668, 127, 15, 102]


In [ ]:
# Convertimos a dataset
dataset_2 = tf.data.Dataset.from_tensor_slices(sequences)

## Predicción

Dividimos las secuencias en entrada y salida (target):

In [ ]:
seq_length = 10  # Longitud de cada secuencia
sequences = dataset_2.batch(seq_length + 1, drop_remainder=True)

def split_input_target(chunk):
    input_text = chunk[:-1]  # Todo menos el último
    target_text = chunk[1:]  # Todo menos el primero
    return input_text, target_text

dataset_2 = sequences.map(split_input_target)


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


In [ ]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset_2 = dataset_2.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

## Entrenamos el modelo palabra a palabra

In [ ]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(
            rnn_units,
            return_sequences=True,
            return_state=True
        )
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        # Embedding layer ensures 3D tensor (batch_size, seq_length, embedding_dim)
        x = self.embedding(inputs, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x


In [ ]:
vocab_size = len(word_index) + 1  # Incluye 1 para el índice 0 (padding, si se usa)
embedding_dim = 256  # Dimensión del embedding (puedes ajustar según el experimento)
rnn_units = 1024  # Número de unidades en la GRU (también ajustable)

model_2 = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units
)


In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)


In [ ]:
EPOCHS = 20

In [ ]:
# for input_example, target_example in dataset_2.take(1):
#     print(f"Entrada (IDs): {input_example.numpy()}")
#     print(f"Objetivo (IDs): {target_example.numpy()}")

In [ ]:
model_2.compile(optimizer='adam', loss=loss)
history = model_2.fit(dataset_2, epochs=EPOCHS, callbacks=[checkpoint_callback])


Epoch 1/20
289/289 [==============================] - 17s 53ms/step - loss: 7.0909
Epoch 2/20
289/289 [==============================] - 16s 53ms/step - loss: 6.3032
Epoch 3/20
289/289 [==============================] - 16s 53ms/step - loss: 5.8748
Epoch 4/20
289/289 [==============================] - 16s 54ms/step - loss: 5.4771
Epoch 5/20
289/289 [==============================] - 15s 52ms/step - loss: 5.0090
Epoch 6/20
289/289 [==============================] - 16s 54ms/step - loss: 4.4585
Epoch 7/20
289/289 [==============================] - 15s 52ms/step - loss: 3.8888
Epoch 8/20
289/289 [==============================] - 15s 52ms/step - loss: 3.3593
Epoch 9/20
289/289 [==============================] - 16s 53ms/step - loss: 2.8779
Epoch 10/20
289/289 [==============================] - 15s 52ms/step - loss: 2.4419
Epoch 11/20
289/289 [==============================] - 16s 52ms/step - loss: 2.0601
Epoch 12/20
289/289 [==============================] - 16s 52ms/step - loss: 1.7350
E

In [ ]:
def prepare_seed(seed_text, tokenizer, seq_length):
    # Convierte las palabras de la semilla en IDs
    seed_seq = tokenizer.texts_to_sequences([seed_text])[0]
    # Ajusta la longitud agregando ceros al inicio si es necesario
    seed_seq = tf.keras.preprocessing.sequence.pad_sequences(
        [seed_seq], maxlen=seq_length, padding='pre'
    )
    return seed_seq


In [ ]:
def generate_text(model, tokenizer, seed_text, seq_length, num_words, temperature=1.0):
    generated_text = seed_text
    seed_seq = prepare_seed(seed_text, tokenizer, seq_length)

    for _ in range(num_words):
        # Predice las probabilidades de la siguiente palabra
        predictions = model(seed_seq, training=False)
        predictions = predictions[:, -1, :]  # Obtén las predicciones de la última palabra

        # Ajusta las probabilidades usando la temperatura
        predictions = tf.nn.softmax(predictions / temperature).numpy()

        # Selecciona la siguiente palabra basada en las probabilidades
        next_word_id = np.random.choice(len(predictions[0]), p=predictions[0])
        next_word = tokenizer.index_word.get(next_word_id, "<unk>")

        # Agrega la palabra generada al texto
        generated_text += " " + next_word

        # Actualiza la semilla
        seed_seq = np.append(seed_seq[0], next_word_id)[-seq_length:]
        seed_seq = np.expand_dims(seed_seq, axis=0)

    return generated_text


In [ ]:
seed_text = "To be or not to be"
generated = generate_text(
    model=model_2,
    tokenizer=tokenizer,
    seed_text=seed_text,
    seq_length=10,  # Debe coincidir con `seq_length` usado en el entrenamiento
    num_words=50,   # Número de palabras a generar
    temperature=1.0 # Experimenta con valores <1 (más conservador) o >1 (más creativo)
)
print("Texto generado:")
print(generated)


Texto generado:
To be or not to be his valiant on one person runs but sore himself on him or my brother if that knew your grace i'll speak that cold lord king henry vi peace for the king my mind is an heir that went my lord by once and wilt this cut from my end by


In [ ]:
translated = translator.translate(generated, src='en', dest='es').text

# Mostrar texto traducido
print("\nTexto traducido al español:")
print(translated)


Texto traducido al español:
Ser o no ser su valiente en una persona corre, pero se adhiere a él o a mi hermano si eso supiera tu gracia, hablaré ese frío Señor Rey Henry VI Paz para el Rey Mi mente es un heredero que fue mi Señor una vezy marchitar este corte de mi final por


En este caso podemos ver que, aunque el texto generado no es perfectamente lógico, las palabras están organizadas en frases que tienen una estructura gramatical razonable en varios tramos. Esto indica que el modelo ha captado patrones frecuentes en el texto de entrenamiento.

 Algunas partes del texto tienen sentido ambiguo o parecen fuera de contexto, como "runs but sore himself on him" o "an heir that went my lord by once". Esto refleja que el modelo aún tiene dificultades para generar contenido con sentido narrativo completo.

 El texto no se genera en forma de verso, como si lo esta el texto de entramiento.


## Evaluamos el modelo

In [ ]:
def generate_multiple_fragments(model, tokenizer, seq_length, num_words, num_fragments, temperature):
    fragments = []
    for _ in range(num_fragments):
        # Generar una semilla aleatoria de palabras
        random_seed = " ".join(np.random.choice(list(tokenizer.word_index.keys()), size=seq_length))
        # Generar texto basado en la semilla
        generated_text = generate_text(
            model=model,
            tokenizer=tokenizer,
            seed_text=random_seed,
            seq_length=seq_length,
            num_words=num_words,
            temperature=temperature
        )
        fragments.append(generated_text)
    return fragments


In [ ]:
# Generar fragmentos al azar
num_fragments = 5
num_words = 50
seq_length = 10
temperature = 1.0  # Puedes ajustar según el análisis solicitado

# Fragmentos para modelo palabra a palabra
fragments_word_to_word = generate_multiple_fragments(
    model=model_2,
    tokenizer=tokenizer,
    seq_length=seq_length,
    num_words=num_words,
    num_fragments=num_fragments,
    temperature=temperature
)

print("Fragmentos generados (Palabra a palabra):")
for i, fragment in enumerate(fragments_word_to_word, 1):
    print(f"Fragmento {i}:\n{fragment}\n")


Fragmentos generados (Palabra a palabra):
Fragmento 1:
owe corslet fellow'st afeard aiding wonderful cleaving reigning purgatory confounds the course hastings and montague that will be piece of thee take on thee in thy and cross and bring we make deadly this second senator help back away and we'll aid thee here who lost there this a prince mine for thou art an executioner king richard iii farewell

Fragmento 2:
valance two bides lineal catcher beginning miseries follower tinkers bluntly we'll have the most of them 3 king bolingbroke so many a french crown to conquest to tell it how should not he spoke first citizen he cannot come menenius if you do i but by your grace by my poor complaint late he should for your art king lewis

Fragmento 3:
star lordship sojourn ensnareth masquing wholesomest myself immaculate methought cures in their nicely like an easy fills and maids by one royal blind to our daughter and be obedient well then thou art a villain romeo what am the wench that juliet i

Ahora vamos a evaluarlo modificando la temperatura y la longitud de secuencia

In [ ]:
def generate_text(
    model, tokenizer, seed_text, seq_length, num_words, temperature=1.0
):
    """Genera texto palabra a palabra con un modelo entrenado."""
    input_text = seed_text
    for _ in range(num_words):
        # Tokenización y preprocesamiento
        token_list = tokenizer.texts_to_sequences([input_text])[0]
        token_list = token_list[-seq_length:]  # Usar solo los últimos `seq_length` tokens
        token_list = tf.keras.preprocessing.sequence.pad_sequences(
            [token_list], maxlen=seq_length, padding='pre'
        )

        # Predicción
        predictions = model.predict(token_list, verbose=0)

        # Select the logits for the next word prediction (last timestep)
        predictions = predictions[:, -1, :]  # Reshape to [1, vocabulary_size]

        predictions = predictions / temperature  # Ajustar temperatura
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Decodificar la palabra predicha
        output_word = tokenizer.index_word.get(predicted_id, "")
        if output_word == "":  # Si no se encuentra, detener
            break
        input_text += " " + output_word

    return input_text

# Parámetros iniciales
seed_text = "To be or not to be"
seq_lengths = [5, 10, 20]  # Diferentes longitudes de secuencia
temperatures = [0.5, 1.0, 1.5]  # Diferentes temperaturas
num_words = 50  # Palabras generadas

# Evaluar combinaciones
for seq_length in seq_lengths:
    print(f"\n=== Evaluando longitud de secuencia: {seq_length} ===")
    for temp in temperatures:
        print(f"\n** Temperatura: {temp} **")
        generated_text = generate_text(
            model=model,
            tokenizer=tokenizer,
            seed_text=seed_text,
            seq_length=seq_length,
            num_words=num_words,
            temperature=temp
        )
        print(generated_text)



=== Evaluando longitud de secuencia: 5 ===

** Temperatura: 0.5 **


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


To be or not to be and shall from are how you they and are good now is the him are was was come are good now is and she from now was now and come they and from now which o and how at and do now and from now was and now then at

** Temperatura: 1.0 **
To be or not to be and thy well how was are do which now that and it at more now they and are how and from are here now and on was now now and how from their and lord well or now and come how for the the but are at which now was

** Temperatura: 1.5 **
To be or not to be and shall with by shall he it as is the with from and they now now and from now was and lord come now to the the thy well they they at then to the the but by king him he what is the thou at was lord and your

=== Evaluando longitud de secuencia: 10 ===

** Temperatura: 0.5 **
To be or not to be and he they and how from now was now and from are here now and he and lord now then their and how from now now and come how and do their and how from now and they well come how at w

**Conclusiones:**

* Temperatura = 0.5:

El modelo tiende a ser más conservador, eligiendo palabras con alta probabilidad en lugar de explorar opciones menos frecuentes.
Esto resulta en texto repetitivo, con frases como "and from now" o "now and how" repitiéndose en múltiples instancias.
La coherencia general es moderada, pero el texto carece de variedad y creatividad.

* Temperatura = 1.0:

Se alcanza un balance entre creatividad y coherencia.
Hay más variedad en las palabras generadas, aunque algunas combinaciones no tienen sentido completo.

* Temperatura = 1.5:

La generación se vuelve muy creativa pero menos coherente.
Frases poco comprensibles.

* Secuencia = 5:

La información de contexto es limitada, lo que provoca un texto más repetitivo y menos contextualizado.
Frases como "and are good now" se repiten con poca relación entre las palabras generadas.

* Secuencia = 10:

El modelo tiene un contexto más amplio, lo que mejora la coherencia del texto.
Se observa mayor diversidad, pero aún persisten repeticiones y falta de una narrativa clara.

* Secuencia = 20:

Un contexto más largo permite al modelo generar frases más variadas y conectadas.



---



La combinación de temperatura = 1.0 y una longitud de secuencia mayor (10 o 20) produce los textos más equilibrados, con cierta diversidad y un nivel aceptable de coherencia.
Temperatura alta (>1.0) puede ser útil para explorar creatividad, pero genera frases menos comprensibles.



# Comparación y conclusiones sobre ambos modelos

**Modelo Carácter a Carácter:**

**Ventajas:**
* Sigue patrones detallados a nivel de
caracteres, incluyendo ortografía, puntuación y estilo.
* Útil para generar el texto en forma de verso.

**Desventajas:**
* Puede generar palabras inexistentes o ininteligibles debido a la falta de una comprensión semántica más amplia.
* Coherencia limitada en frases largas, ya que no tiene un concepto explícito de "palabra".


---


**Modelo Palabra a Palabra:**

**Ventajas:**
* Genera texto con mayor coherencia y sentido lógico.
* Es menos propenso a errores de ortografía.

**Desventajas:**
* No genera el texto siguiendo la forma de verso.
* No conserva patrones literarios.

---

**Creatividad:**
* Carácter a Carácter:
Muestra creatividad en las combinaciones de letras y patrones inesperados.
Sin embargo, esta creatividad puede llevar a incoherencia y palabras inventadas.
* Palabra a Palabra:
Genera textos más predecibles, aunque todavía variados, especialmente con temperaturas más altas.
Produce combinaciones menos arriesgadas pero más comprensibles.
---

**Coherencia:**
* Carácter a Carácter:
Frases cortas pueden ser coherentes, pero se pierde sentido en secuencias más largas debido a la falta de semántica.
Con temperatura = 1.0, logra el mejor equilibrio entre creatividad y sentido.

* Palabra a Palabra:
Tiene una clara ventaja en coherencia. Incluso con secuencias largas, los textos tienen mayor lógica.
---
**Repetición:**
* Carácter a Carácter:
Tiende a repetir patrones y estructuras a nivel de letras o sílabas.
* Palabra a Palabra:
Puede repetir palabras, pero la repetición es menos frecuente.
---
**Parámetros:**
* Carácter a Carácter:
Muy sensible a la longitud de secuencia: secuencias más largas generan texto más fluido, aunque con riesgos de incoherencia.
La temperatura influye mucho en la creatividad, pero temperaturas muy altas tienden a producir texto caótico.
* Palabra a Palabra:
La longitud de secuencia mejora la capacidad de mantener contexto, pero incluso con secuencias cortas genera textos aceptables.
La temperatura afecta la diversidad de palabras pero no compromete tanto la coherencia.

---
**Conclusión General**

Si el objetivo es la creatividad o textos que imiten un patrón específico, el modelo carácter a carácter es más adecuado, aunque requiere ajustes para evitar incoherencias.
Si el objetivo es generar textos comprensibles y coherentes el modelo palabra a palabra es claramente superior..





